In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

print("🧽 Cleaning final dataset...")

input_path = r"C:\Users\Ayush Ahlawat\OneDrive\Documents\Public Comment Analysis\public-comment-analysis\dataset\final_merged_dataset.csv"
output_path = r"C:\Users\Ayush Ahlawat\OneDrive\Documents\Public Comment Analysis\public-comment-analysis\dataset\final_merged_dataset_cleaned.csv"

df = pd.read_csv(input_path)

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join(word for word in text.split() if word not in stop_words)
    text = " ".join(lemmatizer.lemmatize(word) for word in text.split())
    return text

df["cleaned_comment_text"] = df["comment_text"].apply(clean_text)

df.to_csv(output_path, index=False)

print("🎉 Preprocessing Completed!")
print("Saved cleaned file:", output_path)
print("Rows:", len(df))


[nltk_data] Downloading package stopwords to C:\Users\Ayush
[nltk_data]     Ahlawat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ayush
[nltk_data]     Ahlawat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


🧽 Cleaning final dataset...
🎉 Preprocessing Completed!
Saved cleaned file: C:\Users\Ayush Ahlawat\OneDrive\Documents\Public Comment Analysis\public-comment-analysis\dataset\final_merged_dataset_cleaned.csv
Rows: 5345
